# Download the data

In [1]:
from datasets import load_dataset

# If the dataset is gated/private, make sure you have run huggingface-cli login
hf_dataset = load_dataset("datadrivenscience/ship-detection")

Resolving data files:   0%|          | 0/257 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/386 [00:00<?, ?it/s]

In [2]:
hf_dataset

DatasetDict({
    train: Dataset({
        features: ['image', 'objects'],
        num_rows: 256
    })
    test: Dataset({
        features: ['image', 'objects'],
        num_rows: 385
    })
})

In [7]:
import json
import os

import pyarrow as pa
import pyarrow.parquet as pq
from datasets import load_dataset

# Specify the base directory for storing the data
base_dir = (
    "/home/strickvl/coding/zenml-projects/end-to-end-computer-vision/data"
)

# Create directories to store the extracted images
os.makedirs(os.path.join(base_dir, "train_images"), exist_ok=True)
os.makedirs(os.path.join(base_dir, "test_images"), exist_ok=True)


# Function to process each split (train and test)
def process_split(split):
    data = []
    for idx, example in enumerate(hf_dataset[split]):
        image = example["image"]
        objects = example["objects"]

        # Save the image to the local filesystem
        image_filename = f"{split}_{idx}.jpg"
        image_path = os.path.join(base_dir, f"{split}_images", image_filename)
        with open(image_path, "wb") as f:
            f.write(image.tobytes())

        # Create a dictionary with image filename and bounding boxes
        data.append(
            {"image_filename": image_filename, "bboxes": objects["bbox"]}
        )

    return data


# Process train and test splits
train_data = process_split("train")
test_data = process_split("test")

# Save the data as JSON files
with open(os.path.join(base_dir, "train_data.json"), "w") as f:
    json.dump(train_data, f)

with open(os.path.join(base_dir, "test_data.json"), "w") as f:
    json.dump(test_data, f)

# Save the data as Parquet files
train_table = pa.Table.from_pylist(train_data)
pq.write_table(train_table, os.path.join(base_dir, "train_data.parquet"))

test_table = pa.Table.from_pylist(test_data)
pq.write_table(test_table, os.path.join(base_dir, "test_data.parquet"))

KeyboardInterrupt: 

In [14]:
import json

# Create a dictionary to store the categories for each split
categories_dict = {
    "train": [
        example["objects"]["categories"] for example in hf_dataset["train"]
    ],
    "test": [
        example["objects"]["categories"] for example in hf_dataset["test"]
    ],
}

# Save the categories as a JSON file
with open("categories.json", "w") as json_file:
    json.dump(categories_dict, json_file)

print("Categories saved as categories.json")

Categories saved as categories.json


In [13]:
# Print the dataset features
print("Train features:", hf_dataset["train"].features)
print("Test features:", hf_dataset["test"].features)

Train features: {'image': Image(mode=None, decode=True, id=None), 'objects': {'bbox': Sequence(feature=Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), length=-1, id=None), 'categories': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}}
Test features: {'image': Image(mode=None, decode=True, id=None), 'objects': {'bbox': Sequence(feature=Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), length=-1, id=None), 'categories': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}}


In [7]:
import os

from PIL import Image

for split in ["train", "test"]:
    # Create a directory to store the extracted images
    output_dir = f"/home/strickvl/coding/zenml-projects/end-to-end-computer-vision/data/ships/{split}"
    os.makedirs(output_dir, exist_ok=True)

    # Specify the maximum size for the resized images
    max_size = (1000, 1000)

    # Increase the maximum image size limit
    Image.MAX_IMAGE_PIXELS = 1_000_000_000  # Adjust the limit as needed

    # Iterate over the examples in the specified split
    for i, example in enumerate(hf_dataset[split]):
        # Get the image data from the example
        image_data = example["image"]

        # Resize the image while maintaining the aspect ratio
        image_data.thumbnail(max_size)

        # Generate a unique filename for the image
        image_filename = f"image_{i}.png"

        # Save the resized image data to the output directory
        image_path = os.path.join(output_dir, image_filename)
        image_data.save(image_path)

    print(f"Images for '{split}' extracted and resized successfully.")

Images for 'test' extracted and resized successfully.


# Setup dataset

In [10]:
import fiftyone as fo

name = "ships"

# Create the dataset
dataset = fo.Dataset.from_dir(
    dataset_dir=output_dir,
    dataset_type=fo.types.ImageDirectory,
    name=name,
)

# View summary info about the dataset
print(dataset)

# Print the first few samples in the dataset
print(dataset.head())

   0% ||----------------|   1/256 [5.2ms elapsed, 1.3s remaining, 191.7 samples/s] 

 100% |█████████████████| 256/256 [24.8ms elapsed, 0s remaining, 10.3K samples/s]  
Name:        ships
Media type:  image
Num samples: 256
Persistent:  False
Tags:        []
Sample fields:
    id:       fiftyone.core.fields.ObjectIdField
    filepath: fiftyone.core.fields.StringField
    tags:     fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
[<Sample: {
    'id': '6627738801d54b9ea7b3c377',
    'media_type': 'image',
    'filepath': '/home/strickvl/coding/zenml-projects/end-to-end-computer-vision/data/ships/image_0.png',
    'tags': [],
    'metadata': None,
}>, <Sample: {
    'id': '6627738801d54b9ea7b3c378',
    'media_type': 'image',
    'filepath': '/home/strickvl/coding/zenml-projects/end-to-end-computer-vision/data/ships/image_1.png',
    'tags': [],
    'metadata': None,
}>, <Sample: {
    'id': '6627738801d54b9ea7b3c379',
    'media_type': 'image',
    'filepath': 

# Make initial predictions using pre-trained Yolov8 model

In [11]:
# Suppress Ultralytics logging
import os

os.environ["YOLO_VERBOSE"] = "False"

import fiftyone as fo
from ultralytics import YOLO

In [ ]:
# YOLOv8
model = YOLO("yolov8s.pt")

dataset.apply_model(model, label_field="boxes")

session = fo.launch_app(dataset)

In [19]:
# Step 3: Send samples to Label Studio

# A unique identifier for this run
anno_key = "shipdet1"

label_schema = {
    "new_ground_truth": {
        "type": "detections",
        "classes": ["ship"],
    },
}

dataset.annotate(
    anno_key,
    backend="labelstudio",
    label_schema=label_schema,
    launch_editor=True,
    url="http://localhost:8080",
    api_key="9896fa7abd15277ab9046842469fe3c2ec17e0bb",
)
print(dataset.get_annotation_info(anno_key))

# Step 4: Perform annotation in Label Studio and save the tasks

The backend 'labelstudio' does not support attributes. Provided attributes will be ignored.
Uploading media to Label Studio...
Upload complete
Launching editor at 'http://localhost:8080/projects/8'...
{
    "key": "shipdet1",
    "version": "0.23.8",
    "timestamp": "2024-04-23T08:51:56.926945",
    "config": {
        "cls": "fiftyone.utils.labelstudio.LabelStudioBackendConfig",
        "type": "annotation",
        "method": "labelstudio",
        "name": "labelstudio",
        "label_schema": {
            "new_ground_truth": {
                "type": "detections",
                "classes": [
                    "ship"
                ],
                "attributes": {},
                "existing_field": false
            }
        },
        "media_field": "filepath",
        "url": "http://localhost:8080",
        "project_name": null
    }
}


In [20]:
import fiftyone as fo

# Step 5: Merge annotations back into FiftyOne dataset

dataset = fo.load_dataset(name)
dataset.load_annotations(anno_key)

# Load the view that was annotated in the App
view = dataset.load_annotation_view(anno_key)
session = fo.launch_app(view=view)

Download complete
Loading labels for field 'new_ground_truth'...
 100% |███████████████████| 15/15 [12.7ms elapsed, 0s remaining, 1.2K samples/s] 


In [1]:
import fiftyone as fo

fo.launch_app()

Dataset:     -
Session URL: http://localhost:5151/